### Inspiration to work with SMALL DATA

Being new to Datascience, I was fascinated with the dearth of data in Google Big Data repository. The intro courses on SQL gave hands on with the couple of databases that are hosted on the Google cloud. 

To access the Bigdata databases, there is the Big Query library with many wrapper functions. I wanted to dive a bit deeper and setup the Bigquery library in my local machine VS Code environment. But library was not installing and working as expected. There could be some environment conflicts, which I was unable to figure out. I also wanted to learn SQL query more deeply, and keep notes of my learnings.

When I searched around there were many notebooks which used Big query only. I felt that, before accessing more complex database, baby steps needs to be taken by working with traditional SMALL DATA. So the notebook came into existence...

(I used this notebook as an opportunity to refresh my memory too...)

### What to expect? 

1) 3 ways to access SQLite databases, create tables, insert and read data

2) This notebook is like a memory hooks, or heuristic to create quick and dirty databases, and ways to access it. Those who have completed the Intro SQL and Advanced SQL in Kaggle, looking for setting up databases, that they can query themselves 

3) To the point commands to 

    Creating Database files
    
    Setup connection to database
    
    Creating tables 
    
    Inserting data into tables
    
    Querying the tables
    
    Wrapper functions

### What not to expect?

    Indepth consideration of the libraries mentioned

    Advanced SQL queries or receipes

Lets Query in...Come on

Acknowledgements

https://dataset.readthedocs.io/en/latest/

https://docs.python.org/3/library/sqlite3.html

https://docs.sqlalchemy.org/en/14/core/connections.html

Thanks to Tutorialspoint for their information on SQLite implementations...

In [1]:
%pip install dataset

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
from sqlite3 import Error #

import sqlalchemy # This is required for running the queries directly on the notebook

import dataset # Bringing in the dataset library


# Method 1: Using dataset Library

In [3]:
db1 = dataset.connect('sqlite:///exampleDB1.db') # creating and connecting to database library

In [4]:
#Creating table with dataset library. It is very similar to Pandas Dataframe.
db1['mytable'] # using the API method to create and instantiate the new table  

<Table(mytable)>

In [5]:
#Creating instance of the database table
tbl = db1['mytable']

In [6]:
#Method 1 to Insert data. 
tbl.insert({ 'text' : 'Teddy','status' : 'Toy' })

1

In [7]:
tbl.insert({ 'text' : 'Bear','status' : 'Animal' })

2

In [8]:
#Method 2 to insert data.
m_id = db1['mytable'].insert({ 'text' : 'mailing',
                                         'status' : 'Done' })
m_id = db1['mytable'].insert({ 'text' : 'writing',
                                         'status' : 'Progress' })

In [9]:
#Listing the tables in the database
db1.tables

['mytable']

In [10]:
#Listing the columns in the selected table
tbl.columns

['id', 'text', 'status']

In [11]:
#Now comes the interesting part, the way you query the table contents. Use the query function... 
statement = 'SELECT * FROM mytable' #the sql query that you use in database is used here

for row in db1.query(statement):
    print(row) #output is an Ordered dict

OrderedDict([('id', 1), ('text', 'Teddy'), ('status', 'Toy')])
OrderedDict([('id', 2), ('text', 'Bear'), ('status', 'Animal')])
OrderedDict([('id', 3), ('text', 'mailing'), ('status', 'Done')])
OrderedDict([('id', 4), ('text', 'writing'), ('status', 'Progress')])


There you go, database is setup, table is created, data is inserted and it is queried. You will find that using dataset library in your projects for web scraping, storing information that needs to be read by scripts or programs written in other languages much easier.

# METHOD 2: sqlalchemy engine... 

In [12]:
#Creating an sql engine to query using python command
sqlalchemy.create_engine('sqlite:///exampleDB1.db')

Engine(sqlite:///exampleDB1.db)

Then we start using the sql magic for querying the table directly. Then let the engine to establish connection with the database

In [13]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [14]:
%sql sqlite:///exampleDB1.db

'Connected: @exampleDB1.db'

We can write the query directly in the Cell to view the output below. Biggest advantage is, it simulates the database output itself. 

In [15]:
%%sql 

SELECT *
FROM mytable

 * sqlite:///exampleDB1.db
Done.


id,text,status
1,Teddy,Toy
2,Bear,Animal
3,mailing,Done
4,writing,Progress


### We can create tables using the engine as shown below
Note, multiple queries cannot be executed in the same cell. Also the using the magic %sql alters how the cell behaves, even the #ed comments throw error. 


### Creating Table

In [16]:
%%sql 
CREATE TABLE DEPT
       (DEPTNO integer,
        DNAME VARCHAR(14),
        LOC VARCHAR(13) )

 * sqlite:///exampleDB1.db
Done.


[]

### Inserting data into created table in next 3 cells

In [17]:
%%sql
INSERT INTO DEPT VALUES (10, 'ACCOUNTING', 'NEW YORK')

 * sqlite:///exampleDB1.db
1 rows affected.


[]

In [18]:
%%sql
INSERT INTO DEPT VALUES (20, 'MARKETING', 'DALLAS')

 * sqlite:///exampleDB1.db
1 rows affected.


[]

In [19]:
%%sql
INSERT INTO DEPT VALUES (30, 'MANAGEMENT', 'WASHINGTON')

 * sqlite:///exampleDB1.db
1 rows affected.


[]

### Querying the data from the table

In [20]:
%%sql

SELECT *
FROM DEPT

 * sqlite:///exampleDB1.db
Done.


DEPTNO,DNAME,LOC
10,ACCOUNTING,NEW YORK
20,MARKETING,DALLAS
30,MANAGEMENT,WASHINGTON


The jupyter notebook turned into SQL querying shell, with the help of SQLAlchemy. No wonder it is called "Alchemy". Think about the power the library provides, to those who want to show the results of the queries in structured way. Even to document the examples of the unique queries you learn, or try can be super easy. 

# Method 3: SQLite library

In [21]:
#Connecting with sqlite3 library connection gives flexibility to create wrappers as below
conn = sqlite3.connect('exampleDB1.db')
c = conn.cursor()

In [22]:
#Wrapper Functions using sqlite3 library methods
def create_table():
    c.execute('CREATE TABLE IF NOT EXISTS Rone (Number REAL, Name TEXT)')
 
def data_entry():
    number = 1234
    name = "Kagglers_Galore"
    c.execute("INSERT INTO Rone (Number, Name) VALUES(?, ?)", (number, name))
    conn.commit()

def query_table():
    return c.execute("SELECT DEPTNO, DNAME FROM DEPT WHERE DEPTNO = 10").fetchall()

def query_gen(query_input): #The query can be modified
    return c.execute(query_input).fetchall() #Each row is a list inside a list

def ret_byRow(query_input): #This method ensures the memory is not used heavily by loading by the row
    cursor = c.execute(query_input)
    for row in cursor:
              print(row) # each row is a seperate list

In [23]:
#Creating table
create_table()

In [24]:
#inserting data
data_entry()

In [25]:
query_table()

[(10, 'ACCOUNTING')]

In [26]:
q1='SELECT * FROM mytable'
query_gen(q1) #Output will be list of lists

[(1, 'Teddy', 'Toy'),
 (2, 'Bear', 'Animal'),
 (3, 'mailing', 'Done'),
 (4, 'writing', 'Progress')]

In [27]:
q2 = 'SELECT * FROM mytable'
ret_byRow(q2) #Here the output is seperate lists. 

(1, 'Teddy', 'Toy')
(2, 'Bear', 'Animal')
(3, 'mailing', 'Done')
(4, 'writing', 'Progress')


3rd method provides the most power, in terms of writing wrapper functions, providing execution environment, and returning python friendly datatypes which in turn can be used in the way you see fit. 
Hope the notebook is helpful....